## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

In [2]:
# Quando não especificamos a URI, o MLFlow considera pastas padrões do sistema
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns'


In [4]:
# mlflow.list_experiments()
mlflow.search_experiments() # Nova pasta criada em 'running_mlflow_experiments'

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1721221803499, experiment_id='0', last_update_time=1721221803499, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Nome do novo experimento (cria uma nova pasta Local)
mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    # Logando os parâmetros no MLFlow (Cria uma nova pasta local para esse exp.)
    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    # Logando as métricas no MLFlow
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    # Logando os dados do modelo em artifact (será salvo em uma pasta local também)
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/07/17 13:11:56 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/685316930121731423/52d6d60e2a6b4229ba48721fc99af07c/artifacts'


In [7]:
# mlflow.list_experiments()
mlflow.search_experiments() # 2 experimentos

[<Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/685316930121731423', creation_time=1721221916149, experiment_id='685316930121731423', last_update_time=1721221916149, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///workspaces/mlops-zoomcamp/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1721221803499, experiment_id='0', last_update_time=1721221803499, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [8]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [11]:
from mlflow.exceptions import MlflowException

# Não é possivel acessar os registros do modelo se estamos usando arquivos locais
try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

In [ ]:
# Para acessar as runs dos arquivos locais, temos que entrar na pasta 'running-mlflow-examples' e rodar 'mlflow ui'
# para que o experimento seja corretamente lido